In [5]:
import os
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, SystemMessage, AIMessage
from langchain.tools import tool


model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite", 
    temperature=0,
    max_output_tokens=1000,
)

In [ ]:
# 도구 호출

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."

model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Seoul?")
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Seoul'}


In [6]:
# 도구(N개)를 모델에 바인딩
model_with_tools = model.bind_tools([get_weather])

# 1. 모델이 도구 호출
messages = [{"role": "user", "content": "What's the weather like in Seoul?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# 2. 도구 실행 및 결과 추가
for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)
    
# 3. 결과를 모델에 전달해 최종 응답 생성
final_response = model_with_tools.invoke(messages)
print(final_response.text)

The weather in Seoul is sunny.


In [10]:
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather in Boston and Tokyo?")

# 모델은 여러가지 도구를 호출 가능
print(response.tool_calls)
# [
#   {'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1'},
#   {'name': 'get_weather', 'args': {'location': 'Tokyo'} , 'id': 'call_2'}
# ]

# 모든 도구 실행(병령과 비동기로 실행 가능)
results = []
for tool_call in response.tool_calls:
    if tool_call['name'] == 'get_weather':
        result = get_weather.invoke(tool_call)
        
    results.append(result)
print(results)

[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': '75438233-ebe0-4a43-a20f-dbd7d9fe3ac1', 'type': 'tool_call'}, {'name': 'get_weather', 'args': {'location': 'Tokyo'}, 'id': '8f90f5ef-e3d7-4112-8533-4932a116eded', 'type': 'tool_call'}]
[ToolMessage(content="It's sunny in Boston.", name='get_weather', tool_call_id='75438233-ebe0-4a43-a20f-dbd7d9fe3ac1'), ToolMessage(content="It's sunny in Tokyo.", name='get_weather', tool_call_id='8f90f5ef-e3d7-4112-8533-4932a116eded')]


In [ ]:
for chunk in model_with_tools.stream("What's the weather in Boston and Tokyo?"):
    # 실시간으로 응답 청크 처리
    for tool_chunk in chunk.tool_call_chunks:
        if name := tool_chunk.get("name"):
            print(f"Tool: {name}")
        if id := tool_chunk.get("id"):
            print(f"ID: {id}")
        if args := tool_chunk.get("args"):
            print(f"Args: {args}")
            
            

Tool: get_weather
ID: 24368420-669a-4760-b97c-e5c641a34957
Args: {"location": "Boston"}
Tool: get_weather
ID: fc2c3f94-ede3-4a59-819e-e3ca599ba707
Args: {"location": "Tokyo"}


In [ ]:
gathered = None

# 청크 스트리밍
for chunk in model_with_tools.stream("What's the weather in Boston?"):
    gathered = chunk if gathered is None else gathered + chunk
print(gathered.tool_calls)

[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'aaad6773-fccf-4c16-a70c-760eec8f6583', 'type': 'tool_call'}]


In [ ]:
from typing_extensions import TypedDict, Annotated

# 구조화된 출력
class MovieDict(TypedDict):
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]
    
model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)

{'title': 'Inception', 'director': 'Christopher Nolan', 'rating': 8.8, 'year': 2010}


In [19]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")
    
    
model_with_structure = model.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'function_call': {'name': 'Movie', 'arguments': '{"title": "Inception", "director": "Christopher Nolan", "rating": 8.8, "year": 2010}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--73ef68dd-d287-4136-b789-2b677d8b065e-0', tool_calls=[{'name': 'Movie', 'args': {'title': 'Inception', 'director': 'Christopher Nolan', 'rating': 8.8, 'year': 2010}, 'id': 'aa7315b4-9994-428f-8ba6-6bbc38b86a0e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 98, 'output_tokens': 35, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}}),
 'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8),
 'parsing_error': None}

In [ ]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in million USD")
    
# 복잡한 중첩 구조화된 출력
model_with_structure = model.with_structured_output(MovieDetails)

title='Inception' year=2010 cast=[Actor(name='Leonardo DiCaprio', role='Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Elliot Page', role='Ariadne')] genres=['Sci-Fi', 'Action', 'Thriller'] budget=160000000.0


In [ ]:
# 멀티모달 출력(제미나이 프리티어에서 멀티모달 지원 X)
response = model.invoke("Create a picture of a cat")
print(response.content_blocks)
# 예시
# [
#   {"type": "text", "text": "Here's a picture of a cat."},
#   {"type": "image", "base64": "...", "mime_type": "image/jpeg"},
# ]

[{'type': 'text', 'text': "Okay, I can help you create a picture of a cat! Since I can't *actually* draw or generate images directly, I'll describe a cat in detail, and you can use that description to visualize it or even use it as a prompt for an AI image generator.\n\nHere are a few options for cat descriptions, ranging from simple to more detailed. Choose the one you like best, or tell me what you'd like to change!\n\n---\n\n**Option 1: A Classic, Cozy Cat**\n\nImagine a **fluffy, ginger tabby cat** curled up in a sunbeam. Its fur is a warm mix of orange, cream, and darker ginger stripes. Its eyes are a soft, **emerald green**, half-closed in contentment. One paw is tucked neatly beneath its chin, and its tail is wrapped loosely around its body. The background is a soft, blurred suggestion of a cozy living room – perhaps a hint of a wooden floor or a soft rug.\n\n---\n\n**Option 2: A Sleek, Mysterious Cat**\n\nPicture a **slender, black cat** with **piercing, golden eyes**. Its fur 

In [ ]:
# 스트림을 이용한 추론 과정 보기
for chunk in model.stream("Why do parrots have colorful feathers?"):
    resoning_steps = [r for r in chunk.content_blocks if r["type"] == "reasoning"]
    print(resoning_steps if resoning_steps else chunk.text)

Parrots have colorful feathers for
 a variety of fascinating reasons, and it's a combination of **evolutionary pressures
 and social signaling**. Here's a breakdown of the main drivers:

**1. Camouflage:**

*   **Blending in with their environment:** Many parrot species live in lush, tropical rainforests with a vibrant and diverse array of green
 foliage, flowers, and fruits. Their bright colors, especially greens, can help them blend seamlessly into this environment, making them harder for predators to spot.
*   **Breaking up their outline:** Even if a parrot isn't perfectly matching
 a specific leaf, a patchwork of colors can break up their body shape, making them less recognizable as prey.

**2. Communication and Social Signaling:**

*   **Mate Attraction:** This is a huge factor. In many species, the
 most vibrant and striking colors are found in males. These bright displays can signal good health, strong genes, and overall fitness to potential mates. A more colorful parrot might b

In [ ]:
# 일반 추론 과정
response = model.invoke("Why do parrots have colorful feathers?")
reasoning_steps = [b for b in response.content_blocks if b["type"] == "reasoning"]
print(" ".join(step["reasoning"] for step in reasoning_steps))


Parrots have colorful feathers for a variety of fascinating reasons, and it's a combination of **evolutionary pressures and social signaling**. Here's a breakdown of the main drivers:

**1. Camouflage:**

*   **Blending in with their environment:** Many parrot species live in lush, tropical rainforests with a vibrant and diverse array of green foliage, flowers, and fruits. Their bright colors, especially greens, can help them blend seamlessly into this environment, making them harder for predators to spot.
*   **Breaking up their outline:** Even if a parrot isn't perfectly matching a specific leaf, a patchwork of colors can break up their body shape, making them less recognizable as prey.

**2. Communication and Social Signaling:**

*   **Mate Attraction:** This is a huge factor. In many species, the most vibrant and striking colors are found in males. These bright displays can signal good health, strong genes, and overall fitness to potential mates. A more colorful parrot might be se

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite", 
    temperature=0,
    max_output_tokens=1000,
)

# 서버쪽 웹 검색 도구 사용
tool = {"type": "web_search"}
model_with_tools = model.bind_tools([tool])

response = model_with_tools.invoke("What was a positive news story from today?")
response.content_blocks

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

# 속도 제한 설정 가능
# 시간당 요청 수만 제한 가능, 요청 크기에 따라 제한은 불가능
rate_limiter = InMemoryRateLimiter(
    requrests_per_minute=0.1,
    check_every_n_seconds=0.1,
    max_bucket_size=10,
)

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite", 
    temperature=0,
    max_output_tokens=1000,
    rate_limiter=rate_limiter,
)

In [ ]:
# 모델 통합의 경우 API 요청을 URL로 구성 가능하고 프록시로도 구성이 가능하다.
# 예시
model = init_chat_model(
    model="MODEL_NAME",
    model_provider="openai",
    base_url="https://api.openai.com/v1",
    api_key="YOUR_API_KEY",
)

from langchain_openai import ChatOpenAI

# 예시
model = ChatOpenAI(
    model="gpt-4o",
    openai_proxy="http://localhost:8080",
)


In [ ]:
# 로그확률(모델의 예측 신뢰도를 측정하고, 계산 효율성과 수치적 안정성을 높이는 데 필수적인 개념) 얻기
model = init_chat_model(
    model="gpt-4o",
    model_provider="openai",
).bind(logprobs=True)


response = model.invoke("Why do parrots talk?")
print(response.response_metadata['logprobs'])

In [18]:
# 토큰 사용량 추적 기능
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

model_1 = init_chat_model(model="google_genai:gemini-2.5-flash-lite")
model_2 = init_chat_model(model="google_genai:gemini-2.5-flash")

callback = UsageMetadataCallbackHandler()
result_1 = model_1.invoke("Hello", config={"callbacks": [callback]})
result_2 = model_2.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata

{'gemini-2.5-flash-lite': {'input_tokens': 2,
  'output_tokens': 10,
  'total_tokens': 12,
  'input_token_details': {'cache_read': 0}},
 'gemini-2.5-flash': {'input_tokens': 2,
  'output_tokens': 137,
  'total_tokens': 139,
  'input_token_details': {'cache_read': 0},
  'output_token_details': {'reasoning': 128}}}

In [ ]:
# 모델 호출 시에 config 매개변수를 통해 추가 구성 가능
# 예시
response = model.invoke(
    "Tell me a joke",
    config={
        "run_name": "joke_generation",              # 이름 지정
        "tags": ["humor", "demo"],                  # 태그 지정
        "metadata": {"user_id": "123"},             # 메타데이터
        "callbacks": [my_callback_handler]          # 콜백 지정
    }
)

In [21]:
# 런타임에 모델 변경 가능

from langchain.chat_models import init_chat_model

configurable_model = init_chat_model(temperature=0.0)

configurable_model.invoke(
    "What's your name",
    config={"configurable": {"model": "google_genai:gemini-2.5-flash-lite"}}
)

configurable_model.invoke(
    "What's your name",
    config={"configurable": {"model": "google_genai:gemini-2.5-flash"}}
)

AIMessage(content='I do not have a name. I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--8dc9ad21-b7f5-49ff-8b74-3b5fb9a881be-0', usage_metadata={'input_tokens': 6, 'output_tokens': 53, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 35}})